In [1]:
import pandas as pd
import os

In [2]:
df = pd.read_csv("temp_data.csv")

In [26]:
df.columns

Index(['SentAt', 'Control', 'Sad', 'Intrusion', 'Encourage', 'FunThings',
       'IgotThis', 'Company', 'Alone', 'Thoughts', 'Safe', 'Appointments',
       'Cancel', 'Laying', 'Sleep', 'Out', 'Eaten', 'Avoid', 'Useful ',
       'Enjoy', 'Pleas', 'Unpl', 'id_var', 'Date', 'Date_2', 'Hour',
       'beep_var', 'day_var', 'consecutive'],
      dtype='object')

In [4]:
print(df.shape)
print(df.isna().sum())
print(df.columns)

(1235, 27)
SentAt             0
StartedAt        691
Control          691
Sad              691
Intrusion        691
Encourage        691
FunThings        691
IgotThis         691
Company          691
Alone            691
Thoughts         691
Safe             691
Appointments     691
Cancel           691
Laying           691
Sleep            691
Qualitysleep    1072
Out              691
Eaten            691
Avoid            691
Useful           691
Enjoy            691
Pleas            691
Pleasant        1076
Unpl             691
Unpleasant      1211
Remarks         1087
dtype: int64
Index(['SentAt', 'StartedAt', 'Control', 'Sad', 'Intrusion', 'Encourage',
       'FunThings', 'IgotThis', 'Company', 'Alone', 'Thoughts', 'Safe',
       'Appointments', 'Cancel', 'Laying', 'Sleep', 'Qualitysleep', 'Out',
       'Eaten', 'Avoid', 'Useful ', 'Enjoy', 'Pleas', 'Pleasant', 'Unpl',
       'Unpleasant', 'Remarks'],
      dtype='object')


In [5]:
df = df.drop(columns = ['StartedAt', 'Qualitysleep', 'Pleasant', 'Unpleasant', 'Remarks'])

In [6]:
#drop all observations hich have all missing except for SentAt
to_drop =  ['Control', 'Sad', 'Intrusion', 'Encourage',
       'FunThings', 'IgotThis', 'Company', 'Alone', 'Thoughts', 'Safe',
       'Appointments', 'Cancel', 'Laying', 'Sleep', 'Out',
       'Eaten', 'Avoid', 'Useful ', 'Enjoy', 'Pleas', 'Unpl']

df = df.dropna(subset = to_drop, how = 'any', axis=0)

In [7]:
#make idvar
df['id_var'] = 1
#make date
df['Date'] = pd.to_datetime(df['SentAt'], format = '%d/%m/%Y %H:%M')
#get the date and hour
df['Date_2'] = [d.strftime("%x") for d in df['Date']]
df['Hour'] = [d.strftime("%H") for d in df['Date']]
#make the beepvar required for graphicalvar
df['first_obs'] = [1 if (x == '09' or x == '10') else 0 for x in df['Hour']]
df['second_obs'] = [2 if (x == '15' or x == '16') else 0 for x in df['Hour']]
df['third_obs'] = [3 if (x == '21' or x == '22') else 0 for x in df['Hour']]
df['beep_var'] = df['first_obs'] + df['second_obs'] + df['third_obs']
#order by date
df = df.sort_values(by='Date')

In [8]:
#make day_var
day_var = pd.DataFrame(pd.date_range(df['Date'].min(), df['Date'].max()))
day_var.columns = ['Date_Range']
day_var['day_var'] = [d.strftime("%x") for d in day_var['Date_Range']]
day_var = day_var.drop('Date_Range', axis = 1).reset_index()
df = df.merge(day_var, how = 'left', left_on = 'Date_2', right_on = 'day_var')

In [9]:
#clean by dropping unnecessary columns and renaming columns
df = df.drop(['first_obs', 'second_obs', 'third_obs', 'day_var'], axis = 1)
#rename index to day_var
df = df.rename(columns = {'index': 'day_var'})
#investigate
df['consecutive'] = df['day_var'].diff()
#because R doesn't start from -
df['day_var'] = df['day_var']  + 1
#conver variables to string
#df['day_var'] = df['day_var'].astype(str)
#df['id_var'] = df['id_var'].astype(str)
#df['beep_var'] = df['beep_var'].astype(str)

In [14]:
df_beep_3 = df[df['beep_var'] == 3]
df_beep_3['consecutive'] = df_beep_3['day_var'].diff()

/var/folders/2g/qx04w6j525l446ptr2wd9w8w0000gn/T/ipykernel_1109/3282089736.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_beep_3['consecutive'] = df_beep_3['day_var'].diff()


In [21]:
df_beep_3.to_csv('df_beep_3.csv')

In [24]:
df_beep_3['consecutive'].head(50)

2      NaN
5      1.0
7      1.0
10     1.0
12     1.0
14     1.0
19     2.0
22     1.0
25     1.0
28     2.0
31     1.0
34     1.0
36     1.0
38     1.0
40     1.0
43     1.0
46     1.0
48     1.0
51     2.0
54     1.0
57     1.0
59     1.0
61     1.0
64     1.0
67     1.0
69     1.0
72     1.0
74     1.0
75     1.0
78     1.0
80     1.0
83     1.0
86     1.0
88     1.0
90     1.0
93     1.0
95     1.0
99     2.0
102    1.0
104    1.0
108    2.0
110    1.0
113    2.0
116    1.0
124    4.0
127    1.0
130    1.0
132    1.0
137    4.0
138    1.0
Name: consecutive, dtype: float64

In [ ]:
df.dtypes

In [32]:
github_issue = df[['SentAt', 'Control', 'Sad', 'Intrusion', 'Encourage', 
         'FunThings', 'IgotThis', 'Alone', 'Thoughts', 
         'Appointments', 'Laying',
         'Out', 'Eaten', 'Avoid', 'Useful ', 'Enjoy', 'day_var', 'beep_var', 'id_var']]

In [33]:
github_issue.to_csv('ESM_data_github_issue.csv')

In [ ]:
print(df.shape)
print(df.isna().sum().sum())
print(df.columns)

In [ ]:

#first = 16/03/2017
#last = 27/04/2018
# just over a year in scope
# note that we have 544 obs but not evenly spaces out

In [ ]:
df.head()